In [ ]:
import os
import tempfile
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Inicializar SparkSession
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("LimpiezaSilver").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

In [ ]:
# Configuración MinIO
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET_BRONCE = os.environ.get("MINIO_BUCKET", "meteo-bronze")
MINIO_BUCKET_SILVER = "meteo-silver"

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

# Crear bucket Silver si no existe
try:
    minio_client.make_bucket(MINIO_BUCKET_SILVER)
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} creado')
except:
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} ya existe')

✅ Datos cargados


In [ ]:
# Cargar archivo más reciente de Bronze
archivo_reciente = None
try:
    print("📥 Buscando archivos Bronce...")
    objects = minio_client.list_objects(MINIO_BUCKET_BRONCE, recursive=True)
    archivos_csv = [obj.object_name for obj in objects if obj.object_name.endswith(".csv")]
    if archivos_csv:
        archivo_reciente = sorted(archivos_csv)[-1]
        print(f"✅ Cargando: {archivo_reciente}")
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, archivo_reciente.split("/")[-1])
        minio_client.fget_object(MINIO_BUCKET_BRONCE, archivo_reciente, temp_file)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        print(f"✅ Cargados {df.count()} registros")
    else:
        print("⚠️ Sin archivos en Bronce")
        df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])
except Exception as e:
    print(f"⚠️ Error: {e}")
    df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])

print(f"\n📊 DataFrame: {df.count()} registros")
df.show(3)

In [ ]:
# Limpieza de datos
print("\n🧹 LIMPIANDO DATOS...")

# Eliminar columnas innecesarias
df = df.drop('pressure', 'uv_level', 'pm25', 'rain_raw', 'wind_raw', 'vibration', 'light')

# Eliminar duplicados
df = df.dropDuplicates()

print(f"\n✅ {df.count()} registros limpios")
df.show(3)

In [ ]:
# Guardar en Silver bucket usando PySpark
print("\n💾 GUARDANDO EN SILVER...")

tabla = archivo_reciente.split('_bronce_')[0] if archivo_reciente and '_bronce_' in archivo_reciente else 'datos'
archivo_silver = f'{tabla}_silver.csv'

try:
    temp_dir = tempfile.gettempdir()
    temp_path_out = os.path.join(temp_dir, "silver_output")
    
    # Usar PySpark para escribir el CSV
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path_out)
    
    # Encontrar el archivo CSV generado por Spark
    import glob
    csv_files = glob.glob(os.path.join(temp_path_out, "*.csv"))
    
    if csv_files:
        csv_file = csv_files[0]
        temp_file_local = os.path.join(temp_dir, archivo_silver)
        
        # Renombrarlo
        os.rename(csv_file, temp_file_local)
        
        # Subir a MinIO
        minio_client.fput_object(MINIO_BUCKET_SILVER, archivo_silver, temp_file_local)
        print(f"✅ {archivo_silver} actualizado en Silver")
        
        # Limpiar
        os.remove(temp_file_local)
        import shutil
        shutil.rmtree(temp_path_out, ignore_errors=True)
        
        print("\n" + "="*70)
        print("✅ LIMPIEZA DE SILVER COMPLETADA")
        print("="*70)
        print(f"📍 Origen: {archivo_reciente}")
        print(f"📍 Destino: meteo-silver/{archivo_silver}")
        print(f"📊 Registros: {df.count()} limpios y sin duplicados")
        print("="*70)
    else:
        print("❌ No se generó archivo CSV")
        
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()